In [13]:
import json
import pandas as pd
import numpy as np
import random
from pathlib import Path


def read_jsonl(file_path):
    # List to store the extracted data
    data = []

    # Read and process the JSONL file
    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line)  # Parse each line as JSON
            custom_id = json_data.get("custom_id")  # Extract the custom_id
            # Access the nested "content" field inside "result"
            content_list = json_data.get("result", {}).get("message", {}).get("content", [])
            for content in content_list:
                if content.get("type") == "text":
                    # Append the custom_id and text content to the data list
                    data.append({"custom_id": custom_id, "text": content.get("text")})

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)
    return df 

#helper lambda function
def extract_vote(text):
    if "[[yes]]" in text and "[[no]]" in text:
        return pd.NA
    elif "[[yes]]" in text:
        return 1
    elif "[[no]]" in text:
        return 0
    else:
        return pd.NA

def extract_data(df):
    #extract the index so we can do a join operation later on 
    df['index'] = df['custom_id'].str.split('-').str[-1].astype(int)

    #extract the batch number for solumn naming 
    num = df['custom_id'].str.split('-').str[-2].astype(int)[0]

    #set index
    df = df.set_index("index").drop(columns=["custom_id"])

    #rename
    df  = df.rename(columns={"text" : f"Text_{num}"})

    #get binary votes from models 
    df[f'Vote_{num}'] = df[f'Text_{num}'].apply(lambda x: extract_vote(x))
    return df


def join_votes(df_list): 
    df = pd.concat(df_list, axis="columns")
    df = df.rename(columns={'Vote_11': 'Vote_10'})
    df = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9", "Vote_10"]]
    df["Proportion_Yes"] = df.mean(axis="columns")
    df["Proportion_Yes_10"] = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9"]].mean(axis='columns')
    
    #
    df["Proportion_Yes_9a"] = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8"]].mean(axis='columns')
    df["Proportion_Yes_9b"] = df[["Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9"]].mean(axis='columns')
    
    df["Proportion_Yes"] = df.mean(axis="columns")

    df["Majority_Vote"] = df["Proportion_Yes"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)

    #Fix cases where this is still a tie!
    df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_10"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)
    random.seed(537)
    if random.random() > 0.5: 
        df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_9a"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)
    else: 
        df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_9b"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)


    df = df.drop(columns=['Proportion_Yes_10', 'Proportion_Yes_9a', 'Proportion_Yes_9b'])

    return df

# Path to the folder
folder_path = Path('../results/claude/')

# Get all file paths in the folder
file_paths = list(folder_path.rglob('*')) 


df_list = []
for path in file_paths:
    df = read_jsonl(path)
    df = extract_data(df)
    df_list.append(df)
df = join_votes(df_list)
df

,Vote_0,Vote_1,Vote_2,Vote_3,Vote_4,Vote_5,Vote_6,Vote_7,Vote_8,Vote_9,Vote_10,Proportion_Yes,Majority_Vote
index,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1.0,1
1,0,1,1,1,1,0,1,1,1,0,0,0.659461,1
2,1,1,1,1,1,1,1,1,1,1,1,1.0,1
3,1,1,1,1,1,1,1,1,0,1,1,0.905791,1
4,1,1,1,1,1,1,1,1,1,1,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,1,1,1,1,1,0,0,0,0,0,1,0.536364,1
2493,0,1,0,0,0,0,0,0,0,0,0,0.094209,0
2494,0,0,0,0,1,0,0,0,0,0,0,0.094209,0


In [14]:
#Finding errors
votes = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9", "Vote_10"]]
print(votes.isna().sum())

Vote_0     0
Vote_1     0
Vote_2     0
Vote_3     0
Vote_4     2
Vote_5     0
Vote_6     0
Vote_7     0
Vote_8     0
Vote_9     1
Vote_10    0
dtype: int64


There are errors in Vote_4 and Vote_9 but overall very few.
We could fix these manually or could just leave them. 
Right now they are set to pd.NA so don't impact our calculations. 

In [15]:
df.mean()

Vote_0            0.694033
Vote_1            0.688426
Vote_2            0.696836
Vote_3            0.682819
Vote_4            0.685772
Vote_5            0.665999
Vote_6            0.671205
Vote_7             0.67521
Vote_8            0.661994
Vote_9            0.668269
Vote_10           0.677213
Proportion_Yes    0.678854
Majority_Vote     0.696035
dtype: object

I sent prompts in 2 formats, 
For votes [0, 4] order of prompting was [[yes]], [[no]], 
For votes [5, 9] oder of prompting was [[no]], [[yes]]
Interestingly, you can see a bias of about 0.02 in responses based on this, so glad I did it! 

In [16]:
ballotq = pd.read_csv('../data/clean_ballot_qs.csv', index_col=0)
result = pd.concat([ballotq, df], axis='columns')
result = result[["Year", "State", "Topic_Areas", "Election", "Percent_Yes", "Pass", "Proportion_Yes", "Majority_Vote"]]
result

,Year,State,Topic_Areas,Election,Percent_Yes,Pass,Proportion_Yes,Majority_Vote
0,2000,Alabama,Civil & Constitutional Law,General,0.600,1,1.0,1
1,2000,Alabama,Judiciary,General,0.570,1,0.659461,1
2,2000,Alabama,Local Government,General,0.630,1,1.0,1
3,2000,Alabama,Human Services,General,0.570,1,0.905791,1
4,2000,Alabama,Bond Measures,General,0.630,1,1.0,1
...,...,...,...,...,...,...,...,...
2492,2014,Wyoming,Education: Higher Ed,General,0.297,0,0.536364,1
2493,2016,Wyoming,Budgets,General,0.563,1,0.094209,0
2494,2020,Wyoming,Energy & Electric Utilities,General,0.510,1,0.094209,0
2495,2022,Wyoming,Judiciary,General,0.390,0,1.0,1


In [18]:
np.sum(result["Majority_Vote"].isna())

np.int64(0)

In [17]:
#Now there are lots of interesting questions we can ask from here! 
#Look at state, Topic areas, 
# How would things change if we filter by General election (some papers suggest that might be a good idea)
# Plot Propotion_Yes (from LLM) against Percent_Yes (from humans)
# Obviously, compare Majority Vote (from LLM) against Pass (from humans)
# Make a confusion matrix 

#Here's a basic start 

no_ties = result.dropna()
print("Among results without ties, percentage of LLM responses aligning with voters")
print(np.mean(no_ties["Pass"]==no_ties["Majority_Vote"]))

##What to do about ties??? Perhaps send to the model with no [[yes]] [[no]] in prompt and manually eval?

Among results without ties, percentage of LLM responses aligning with voters
0.6964357228674409


In [85]:
tp = np.mean((no_ties["Pass"]==1) & (no_ties["Majority_Vote"]==1))
tn = np.mean((no_ties["Pass"]==0) & (no_ties["Majority_Vote"]==0))
fn = np.mean((no_ties["Pass"]==1) & (no_ties["Majority_Vote"]==0))
fp = np.mean((no_ties["Pass"]==0) & (no_ties["Majority_Vote"]==1))
